In [1]:
import torch
from wl_test_l2O import wl_test
from wl_test_final import WLtest
torch.set_printoptions(profile="full")

/mntcephfs/lab_data/hanyizhou/anaconda/wl/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
torch.manual_seed(1)
x1, x2, x3 = torch.randn(1,1), torch.randn(1,1), torch.randn(1,1)
x1,x2,x3

(tensor([[0.6614]]), tensor([[0.2669]]), tensor([[0.0617]]))

In [20]:
info1 = graph_generator('/home/hanyizhou/wl_test/lp_generation/test/answer_eoe_gpt-4o-2024-08-06.lp')
info2 = graph_generator('/home/hanyizhou/wl_test/lp_generation/test/model.lp')
A1 = info1[0]
A2 = info2[0]
f1 = info1[1]#variable
f2 = info2[1]
c1 = info1[2]#constrain
c2 = info2[2]
wltest = WLtest(A1, c1,f1,A2,c2,f2)
print(wltest.test())

Read LP format model from file /home/hanyizhou/wl_test/lp_generation/test/answer_eoe_gpt-4o-2024-08-06.lp
Reading time = 0.00 seconds
: 6 rows, 10 columns, 60 nonzeros
Read LP format model from file /home/hanyizhou/wl_test/lp_generation/test/model.lp
Reading time = 0.00 seconds
: 16 rows, 10 columns, 70 nonzeros
same


In [21]:
c1.shape, c2.shape

(torch.Size([36, 2]), torch.Size([36, 2]))